In [ ]:
# we load the things!

from ngsolve import *
from ngsolve.meshes import MakeStructured3DMesh
from ngsolve.webgui import Draw
from netgen.occ import *
from netgen.csg import *

import matplotlib.pyplot as plt
import scipy.sparse as sp
import numpy as np


from ngsolve.comp import ProxyFunction
from ngsolve.fem import CoefficientFunction

ProxyFunction.__or__ = lambda a, b: InnerProduct(a, b)
CoefficientFunction.__or__ = lambda a, b: InnerProduct(a, b)

In [ ]:
# All functions needed to do convergence study

# custom logspace function
def logspace_custom_decades(start, stop, points_per_decade):
    
    result = []
    current_decade = start
    while current_decade < stop:
        decade_points = np.logspace(np.log10(current_decade), np.log10(current_decade * 10), points_per_decade, endpoint=False)
        result.extend(decade_points)
        current_decade *= 10
    return np.array(result)

# functions for differential operators on manufactured solutions 
coords = [x,y,z]

def JacobianOfCF(cf):
    """ Function to compute the Jacobi Matrix of a vector coefficient function cf """

    Jac_u_3D = CF((
    cf[0].Diff(x), cf[0].Diff(y), cf[0].Diff(z),
    cf[1].Diff(x), cf[1].Diff(y), cf[1].Diff(z),
    cf[2].Diff(x), cf[2].Diff(y), cf[2].Diff(z)
    ), dims=(3, 3))

    return Jac_u_3D

def GGrad(cf):
    """ Function to compute the gradient of a scalar Coefficient Function """
    gg = [cf.Diff(coords[i]) for i in range(mesh.dim)]
    return CF(tuple(gg))


def GCurl(cf):
    """ Function to compute the curl or rot of vec cf using Jacobian """

    if cf.dim == 1: # if the functions is getting handed a scalar field, its to calculate the curl of the rot..
        curl_rot_u = CF((cf.Diff(y), - cf.Diff(x)))
        return curl_rot_u

    elif mesh.dim == 2:
        rot_u = CF(cf[1].Diff(x) - cf[0].Diff(y))
        return rot_u
    
    elif mesh.dim == 3:
        Jac_u = JacobianOfCF(cf)
        curl_u = CF((Jac_u[2,1] - Jac_u[1,2],  
                    Jac_u[0,2] - Jac_u[2,0],  
                    Jac_u[1,0] - Jac_u[0,1]))
        return curl_u
    

def GDiv(cf):
    """ Function to compute the divergence of a vector coefficient function """

    gd = [cf[i].Diff(coords[i]) for i in range(cf.dim)]
    return CF(sum(gd))

def curl_a_times_b(a,b):
    """ Returns curl(a) x b """
    a_Jac = Grad(a)
    
    c0 = a_Jac[2, 1] - a_Jac[1, 2]  # (curl(a))[0]
    c1 = a_Jac[0, 2] - a_Jac[2, 0]  # (curl(a))[1]
    c2 = a_Jac[1, 0] - a_Jac[0, 1]  # (curl(a))[2]      

    cxb0 = c1 * b[2] - c2 * b[1]
    cxb1 = c2 * b[0] - c0 * b[2]
    cxb2 = c0 * b[1] - c1 * b[0]

    curlAtimesB = CF((cxb0, cxb1, cxb2))

    return curlAtimesB

# Functions for plotting, least squares regression fit line for convergence
def reference_line_func(h_values, scaling_factor, slope):

    return scaling_factor * h_values ** slope

def fit_reference_line(h_values, error_values):

    popt, _ = curve_fit(reference_line_func, h_values, error_values, p0=[1, 1])

    scaling_factor, slope = popt
    return scaling_factor, slope

# Functions to calculate h_max
def edge_length(v1, v2, dim):
    return np.sqrt(sum((v1[i] - v2[i])**2 for i in range(dim)))

def squared_distance(v1, v2):
    v1 = np.array(v1)
    v2 = np.array(v2)
    return np.sum((v1 - v2) ** 2)

def cayley_menger_matrix(vertices):
    if len(vertices) != 4:
        raise ValueError("This method is for a tetrahedron, which requires exactly 4 vertices.")

    # Create the Cayley-Menger matrix (5x5)
    C = np.ones((5, 5))
    for i in range(5):
        C[i, i] = 0 

    for i in range(1, 5):
        for j in range(i+1, 5):
            C[i, j] = C[j, i] = squared_distance(vertices[i-1], vertices[j-1])

    return C

def triangle_area(a, b, c):
    s = (a + b + c) / 2  
    return np.sqrt(s * (s - a) * (s - b) * (s - c))

def circumradius_2D(a, b, c):
    area = triangle_area(a, b, c)
    return a * b * c / (4 * area)

def circumradius_3D(vertices):
    C = cayley_menger_matrix(vertices)

    try:
        C_inv = np.linalg.inv(C)
    except np.linalg.LinAlgError:
        raise ValueError("Cayley-Menger matrix is singular or not invertible.")

    M = -2 * C_inv
    circumradius = 0.5 * np.sqrt(M[0, 0])

    return circumradius

def calc_hmax(mesh):
    max_h = 0 
    if mesh.dim == 2:
        for el in mesh.Elements():
            vertices = [mesh[v].point for v in el.vertices]
            a = edge_length(vertices[0], vertices[1], 2)
            b = edge_length(vertices[1], vertices[2], 2)
            c = edge_length(vertices[2], vertices[0], 2)
            circumradius = circumradius_2D(a, b, c)
            max_h = max(max_h, circumradius)
    
    elif mesh.dim == 3:
        for el in mesh.Elements():
            vertices = [mesh[v].point for v in el.vertices]
            circumradius = circumradius_3D(vertices)
            max_h = max(max_h, circumradius)
    
    return max_h

# Functions to create geometries, unstrucuted reentrant corner and structured unit brick 
def createGeometry(hmax):
    """ Given h_max, this function returns an unstructured mesh on the reentrant corner in 3D """

    largeBrick = Box(Pnt(-0.5, -0.5,-0.5), Pnt(0.5, 0.5, 0.5))
    smallBrick = Box(Pnt(-0.5, -0.5,-0.5), Pnt(0, 0, 0))

    reentrantCornerGeo3D = largeBrick - smallBrick
    # Here instead of the following mesh command, I think I need to implement a structured mesh with the manual mesh generation approach.
    mesh = Mesh(OCCGeometry(reentrantCornerGeo3D).GenerateMesh(maxh=hmax))

    return mesh

def createStructuredGeometry(n):
    """ Given n, this functions returns a structured mesh with n elements along each dimnsion on the unit brick """
    mesh =  MakeStructured3DMesh(hexes=False, nx=n, ny=n, nz=n)
    return mesh

def createUnitBrickGeometry(h_init, refineSteps = None):
    unitBrick = OrthoBrick( Pnt(-0.5,-0.5,-0.5), Pnt(0.5,0.5,0.5) )
    geo = CSGeometry()
    geo.Add (unitBrick)
    mesh = geo.GenerateMesh(maxh=h_init)
    if (refineSteps != None):
        for i in range(refineSteps):
            mesh.Refine()
        return Mesh(mesh)
    else:
        return Mesh(mesh)


In [186]:
def hodgeLaplace1Forms(mesh,
                       g,
                       order = 1,
                       C_w = 1):
    
    dS = dx(skeleton  = True, element_vb = BND, definedon=mesh.Boundaries(".*"))
    ProxyFunction.__or__ = lambda a, b: InnerProduct(a, b)
    CoefficientFunction.__or__ = lambda a, b: InnerProduct(a, b)
    def Gn(u): return Cross(n, curl(u))
    def Gt(u): return Cross(n, Cross(u, n))
    
    h_curl = HCurl(mesh, order=order, type1=True)  # For 1-forms, H(curl)
    h_1 = H1(mesh, order=order)     # For 0-forms, H1 space
    fes = h_curl * h_1
    (u, p), (v, q) = fes.TnT()

    B, F = BilinearForm(fes), LinearForm(fes)

    n = specialcf.normal(mesh.dim)
    h = specialcf.mesh_size

    f = CF(GGrad(GDiv(g)) - GCurl(GCurl(g)))

    B += -(curl(u)|curl(v)) * dx
    B += (v|Grad(p)) * dx
    B += (u|Grad(q)) * dx
    B += - (p|q) * dx
    
    B += - (Gt(v)|Gn(u)) * dS
    B += - (Gt(u)|Gn(v)) * dS
    B += - C_w/h * (Gt(u)|Gt(v)) * dS

    F += (f|v) * dx
    
    F +=  - (C_w / h) * (Gt(g)|Gt(v)) * dS
    F +=  - (Gt(g)|Gn(v)) * dS
    F +=  (g*n|q) * dS
    
    with TaskManager():
        B.Assemble(), F.Assemble()
        sol = GridFunction(fes)
        res = F.vec-B.mat * sol.vec
        inv = B.mat.Inverse(freedofs=fes.FreeDofs(), inverse="pardiso")
        sol.vec.data += inv * res

    gf_u , gf_p = sol.components

    p_m =  CF(GDiv(g))
    
    L2_error_u = sqrt(Integrate((gf_u - g)**2, mesh))
    L2_error_p = sqrt(Integrate((gf_p - p_m)**2, mesh))

    return L2_error_u, L2_error_p, gf_u, gf_p
    

In [187]:
# definition of manufactured solutions
#g = CF((x**2*z, 
       # y,
       # x**2))

g = CF((x**2 * sin(z) * cos(y), 
        2 * z**3 * sin(x) * cos(1/3*z),
        -y**2 *-cos(3*z)*sin(x)         ))

# HL_f = CF( ( 2 * sin(z) * cos(y) * (x**2 - 1),
#              sin(x) * ( 4*z**2 * sin(z/3) - 12*z * cos(z/3) + (20/9)*z**3 * cos(z/3) ),
#               2 * cos(3*z) * sin(x) * (5*y**2 - 1) ) )

#f = CF(GCurl(GCurl(g)) - GGrad(GDiv(g)))

#g = CF((z**2, 12*cos(1/3*x), sin(x)+2*y**4))



In [188]:
mesh = createUnitBrickGeometry(0.05)
order = 1
C_w = 10
L2u, L2p, u_sol, p_sol = hodgeLaplace1Forms(mesh, g, order, C_w)

print(L2u, L2p)
print(sqrt(Integrate((g)**2, mesh)))

#0.005078274383628784 0.032772915078126026
#0.046081682227183736

0.046081683717551336 0.11840317670528049
0.046081683717551336


In [189]:
# f = CF(GCurl(GCurl(g)) - GGrad(GDiv(g)))

# Draw(f, mesh)

In [190]:
#Draw(f, mesh)

In [191]:
# error_u = []
# error_p = []
# h_vals = []

# n_list = [12, 14, 16]
# C_w = 10
# order = 1

# for n in n_list:
#     print("Doing n = ", n)
#     mesh = createStructuredGeometry(n)
#     h_val = calc_hmax(mesh)
#     L2_u , L2_p = hodgeLaplace1Forms(mesh, g, order, C_w)
#     h_vals.append(h_val); error_u.append(L2_u) ; error_p.append(L2_p)

# df = pd.DataFrame({
#     'h_max': h_vals,
#     'error_u': error_u,
#     'error_p': error_p
# })



In [192]:
# # Plot the errors against h_max
# plt.figure(figsize=(8, 6))
# plt.plot(df['h_max'], df['error_u'], marker='o', label='L2 error u')
# plt.plot(df['h_max'], df['error_p'], marker='s', label='L2 error p')
# plt.xlabel('h_max')
# plt.ylabel('Error')
# plt.xscale('log')  # Set x-axis to logarithmic scale
# plt.yscale('log')  # Set y-axis to logarithmic scale
# plt.title('Errors vs. h_max')
# plt.legend()
# plt.grid("both")
# plt.show()